**Learning outcomes**
* Define what it means for a gate set to be universal for quantum computing.
* State two universal gate sets for single-qubit quantum computation.

In [5]:
import numpy as np
import pennylane as qml

[qml.Rot documentation](https://docs.pennylane.ai/en/stable/code/api/pennylane.Rot.html) - arbitrary single qubit rotation
![circuit](./images/I.7.1.png)
Even though `Rot` is the most general single-qubit operation, under the hood it's just `RZ` and `RY` gates! This begs the quetion of whether we actually need `RX`- essentially, the answer is no. In fact, in general, as long as we have two rotations out of the set`[RX, RY, RZ]`, we can implement any single-qubit operation; it's just a matter of finding the angles that work, which could be mathematically cumbersome. Together RZ and RY form a universal gate set for single-qubit operations (as do RZ and RX, or RY and RX)


**Codercise I.7.1**
Can you find a set of angles `phi, theta, omega` such that the sequence of gates
`qml.RZ(phi, wires=0)`
`qml.RX(theta, wires=0)`
`qml.RZ(omega, wires=0)`
acts the same as a Hadamard gate (up to global phase)?
Hint:
For convenience, here are the matrix forms for `H` and `RX`:
![circuit](./images/I.7.1.1.png)

Start by determining which angle of the `RX` will give you the correct magnitude of the elements, then use the `RZ` to adjust the signs to give `H` up to global phase.
[Hint](https://quantumcomputing.stackexchange.com/questions/18031/how-to-visualize-hadamard-gate-as-x-z-x-decomposition)

In [13]:
dev = qml.device("default.qubit", wires=1)

#ADJUST THE VALUES OF PHI, THETA, AND OMEGA
phi, theta, omega = np.pi/2, np.pi/2, np.pi/2

@qml.qnode(dev)
def hadamard_with_rz_rx():
    qml.RZ(phi, wires=0)
    qml.RX(theta, wires=0)
    qml.RZ(omega, wires=0)
    return qml.state()

@qml.qnode(dev)
def hadamard_with_rx_rz():
    qml.RX(phi, wires=0)
    qml.RZ(theta, wires=0)
    qml.RX(omega, wires=0)
    return qml.state()

print(hadamard_with_rz_rx())
print(hadamard_with_rx_rz())

[0.-0.70710678j 0.-0.70710678j]
[0.-0.70710678j 0.-0.70710678j]


**Codercise I.7.2.**
 Using the results of the previous exercise, rewrite the following circuit over the gate set `[RZ, RX]` (recall that it is okay for your circuit to work up to a global phase). What is the minimum number of such gates needed to do so?

![circuit](./images/I.7.2.png)

![circuit](./images/I.7.2.1.png)
**Remember** Splot function Y=RX($\pi$)RZ($\pi$), on the circuit we need to apply first RZ($\pi$) gate, subsequently RX($\pi$) gate.

In [14]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def convert_to_rz_rx():

    qml.RZ(np.pi/2, wires=0)
    qml.RX(np.pi/2, wires=0)
    qml.RZ(np.pi*7/4, wires=0)
    qml.RX(np.pi, wires=0)

    return qml.state()

**Codercise I.7.3.** The two gates `H` and `T` are also a universal gate set. By combining just these two gates, we can approximate to arbitrary precision any single-qubit operation (just like we can do with  and )! Write a PennyLane circuit that applies the unitary matrix
![circuit](./images/I.7.3.png)
using 6 `H` and `T` gates altogether. Recall that the matrix form of  can be written up to a global phase as
![circuit](./images/I.7.3.2.png)
Hint:
Consider that the Hadamard is its own inverse. This limits the ordering of how the two operations can be applied in sequence.
Hint:
The common denominator of all the terms holds information about the number of Hadamards you have to use. The largest cumulative phase, and number of terms in the sum should also give you a hint as to how many `T`s are necessary.

Tip. This process is called **quantum circuit synthesis** and is part of the broader subject of quantum compilation. As you will no doubt appreciate after working out this small exercise, designing high-quality, automated compilation tools is an active area of research.

In [ ]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def unitary_with_h_and_t():
    # APPLY ONLY H AND T TO PRODUCE A CIRCUIT THAT EFFECTS THE GIVEN MATRIX
    qml.Hadamard(wires=0)
    qml.T(wires=0)
    qml.Hadamard(wires=0)
    qml.T(wires=0)
    qml.T(wires=0)
    qml.Hadamard(wires=0)

    return qml.state()

Solution. Given that the common denominator is $\sqrt{2}^3$, we can infer that 3 Hadamards have been applied. Since Hadamard is its own inverse, there has to be some number of `T` in between them. Now consider the phases that are present in the matrix elements. There is a phase of $i$, suggesting that there is a pair of `T` gates `TT=S` somewhere. Then, the largest combined phase is $ie^{i\pi \over4}$, so we can figure that there are at most 3 s total. If we know that two of them have to be together to make an `S`, that means the remaining one is flying solo.
There are thus just two combinations (expressed below in matrix multiplication order; the circuits will contain the gates in the opposite order):
* HTTHTH
* HTHTTH
The second one is correct.